# 폐기한 크롤링 코드

>+ 본 파일은 폐기했던 크롤링 코드에 대해서 다룹니다.
>+ 파일 설명 : selium으로 모든 유튜브 페이지를 크롤링하여 정보를 가져오는 코드입니다. <br/>
소요 예상 시간은 100개 채널 데이터 기준으로 약 2~3시간이며,<br/> 
저희 조에서 가져올려고 했던 5개의 카테고리 각 100개의 채널을 가져오는데는 10시간 이상이 들어가서 폐기했습니다.<br/>
>+ Appendix라서 매우 대충 설명할 것입니다.

In [ ]:
# 그냥 모듈 import
import re
import time
import numpy as np
import pandas as pd

from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
# 각 영상 url 한개를 가져와서 스크롤 다운하고 정보 끌어오는 식으로 짰음
# 유튜브 페이지는 일단 브라우저에서 보이지 않으면 크롤링이 안되서 스크롤 하는 식으로 짰음
# 그런데 스크롤 다운 속도가 너무 빠르면 댓글창이 로딩이 안되서 나누기 3으로 천천히 내려서 긁어옴
def video_info(video_urls):
    driver.get(video_urls)
    time.sleep(3)

    while True:
        lastHeight = driver.execute_script("return document.documentElement.scrollHeight/3")
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight/3);")
        time.sleep(1)
        newHeight = driver.execute_script("return document.documentElement.scrollHeight/3")
        if newHeight == lastHeight:
             break

    video_source = BeautifulSoup(driver.page_source, 'html.parser')
    
    trans = lambda x : 0 if '없음' in x else int(x[3:-1].replace(',',''))

    view_count = video_source.find('span',{'class':'view-count'}).get_text()
    view_count = trans(view_count)

    opinion = video_source.find_all('yt-formatted-string',{'class':'ytd-toggle-button-renderer'})

    like_count = trans(opinion[0]['aria-label'])
    dlike_count = trans(opinion[1]['aria-label'])
    
    comment_count = 1
    for find_stop in video_source.find_all('span',{'class':'style-scope yt-formatted-string'}):
        if '댓글이 사용 중지되었습니다.' in find_stop.get_text():
            comment_count = 0
    
    if comment_count:
        comment_count = video_source.find('yt-formatted-string',
                                    {'class':'count-text style-scope ytd-comments-header-renderer'}).find_all('span',{'dir':'auto'})[1].get_text()
        comment_count = int(comment_count)

    video_info = [ view_count, like_count, dlike_count, comment_count ]
    return video_info

In [ ]:
# 채널 정보를 받아서 그 채널 url로 들어가고 
# 동영상 탭에서 최근 30개 영상의 url을 video_info 함수에 파라미터로 넘겨주고 동영상 데이터를 받아옴
# return은 영상 내 정보를 모두 더한값 한 개의 series
def channel_quest(channel):
    channel_url = 'https://www.youtube.com/' + channel[9:]
    driver.get(channel_url)
    time.sleep(3)

    tab_content = driver.find_elements_by_class_name('tab-content')

    for tab in tab_content:
        if (tab.text == '동영상') | (tab.text == 'Videos'):
            tab.click()

    time.sleep(3)

    web = driver.page_source
    source = BeautifulSoup(web, 'html.parser')
    
    try:
        videos = source.find_all('a',{'id':'video-title'})
        video_urls = []
        videos_df = []
        for video in videos:
            video_url = 'https://www.youtube.com' + video['href']
            videos_df.append(video_info(video_url))
            time.sleep(3)
    
        videos_df = pd.DataFrame(videos_df, columns=['view','like','dislike','comments'])
        return videos_df.sum()
    except:
        return pd.Series([0,0,0,0])

In [ ]:
# selium 켜고 타겟 url 보내주고 거기서 250개의 유튜버 정보를 크롤링해옴
# 그걸 파일에 저장함
driver = webdriver.Chrome(executable_path='chromedriver.exe')

influence = "https://kr.noxinfluencer.com/youtube-channel-rank/top-250-kr-film%20%26%20animation-youtuber-sorted-by-subs-weekly"

driver.get(influence)
time.sleep(5)

while True:
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    newHeight = driver.execute_script("return document.body.scrollHeight")
    if newHeight == lastHeight:
         break

web = driver.page_source
source = BeautifulSoup(web, 'html.parser')

channel_href = []
channel_names = []
channel_subscribe = []

for channel in source.find_all('a',{'class','star-avatar'}):
    if channel['href'] != '':
        channel_href.append(channel['href'])
        
for channel_name in source.find_all('span',{'class':'name kol-name'}):
    channel_names.append(channel_name['title'])
    
for channel_subs in source.find_all('td',{'class':'text followerNum with-num'}):
    channel_subscribe.append( channel_subs.find('span',{'class':'num'}).get_text().strip() )
    
channels_info = pd.DataFrame( [x for x in zip(channel_names,channel_subscribe,channel_href) ], columns=['name','subs','url'])
channels_info.to_excel ('channel_info.xlsx')

In [ ]:
# 채널 정보 저장된 파일 불러와서 함수 실행시켜서 모든 채널의 정보(영상 30개 정도의 합)을 가져옴
ani_df = pd.DataFrame(columns=['view','like','dislike','comments'])
channels_info = pd.read_excel('channel_info.xlsx',encoding='utf-8')

driver = webdriver.Chrome(executable_path='chromedriver.exe')
for idx in range(10):
    print('now : ' + channels_info.loc[idx]['url'])
    channel_return = channel_quest(channels_info.loc[idx]['url'])
    ani_df.loc[idx] = channel_return

ani_df

In [ ]:
# selinum 정보
driver.close()
driver.quit()